In [20]:
# https://medium.com/@pierre.beaujuge/classifying-images-from-the-cifar10-dataset-with-pre-trained-cnns-using-transfer-learning-9348f6d878a8
# https://www.mathworks.com/help/deeplearning/ref/densenet201.html

from tensorflow.image import resize

from keras.layers import Lambda, Flatten, Dense, Dropout
from keras.engine import training
from keras.initializers import he_normal
from keras.regularizers import l2
from keras.optimizers import Adam
from keras.optimizers.schedules import ExponentialDecay
from keras.callbacks import ReduceLROnPlateau, EarlyStopping, ModelCheckpoint
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Model
from keras.applications import DenseNet201
from keras import Input

import numpy as np
import matplotlib.pyplot as plt

from load_data import load_data_keras




In [5]:
x_train, x_test, y_train, y_test = load_data_keras()

In [6]:
initializer = he_normal()
model_input = Input(shape=(32, 32, 3))

In [19]:
def DenseNet201_cnn(model_input) -> training.Model:
    resized_images = Lambda(lambda image: resize(image, (224, 224)))(model_input)
    model = DenseNet201(include_top=False,
                                           weights='imagenet',
                                           input_tensor=resized_images,
                                           input_shape=(224, 224, 3),
                                           pooling='max',
                                           classes=1000)
    for layer in model.layers:
        layer.trainable = False
    output = model.layers[-1].output
    flatten = Flatten()
    output = flatten(output)
    layer_256 = Dense(units=256,
                      activation='elu',
                      kernel_initializer=initializer,
                      kernel_regularizer=l2())
    output = layer_256(output)
    dropout = Dropout(0.5)
    output = dropout(output)
    softmax = Dense(units=10,
                    activation='softmax',
                    kernel_initializer=initializer,
                    kernel_regularizer=l2())
    output = softmax(output)
    model = Model(inputs=model_input, outputs=output)
    
    return model

In [12]:
model = DenseNet201_cnn(model_input)

model.compile(optimizer=Adam(learning_rate=1e-4),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [14]:
model_checkpoint_name = "DenseNet201_model_"
batch_size = 128
epochs = 100

In [15]:
lr_schedule = ExponentialDecay(initial_learning_rate=1e-4,
                               decay_steps=10000,
                               decay_rate=0.8)
lr_reduce = ReduceLROnPlateau(monitor='val_accuracy',
                              factor=0.6,
                              patience=2,
                              verbose=1,
                              mode='max',
                              min_lr=1e-8)

early_stop = EarlyStopping(monitor='val_accuracy',
                           patience=3,
                           verbose=1,
                           mode='max')
checkpoint = ModelCheckpoint(model_checkpoint_name+"",
                             monitor='val_accuracy',
                             verbose=1,
                             save_weights_only=False,
                             save_best_only=True,
                             mode='max',
                             save_freq='epoch')

In [18]:
train_datagen = ImageDataGenerator(# rotation_range=20,
                                   # width_shift_range=0.2,
                                   # height_shift_range=0.2,
                                   # shear_range=0.2,
                                   # zoom_range=0.1,
                                   horizontal_flip=True,
                                   #fill_mode='nearest'
)

train_generator = train_datagen.flow(x_train,
                                     y_train,
                                     batch_size=batch_size)
val_datagen = ImageDataGenerator(  # rotation_range=20,
                                   # width_shift_range=0.2,
                                   # height_shift_range=0.2,
                                   # shear_range=0.2,
                                   # zoom_range=0.1,
                                   horizontal_flip=True,
                                   #fill_mode='nearest'
)

val_generator = val_datagen.flow(x_test,
                                 y_test,
                                 batch_size=batch_size)

train_steps_per_epoch = x_train.shape[0] // batch_size
val_steps_per_epoch = x_test.shape[0] // batch_size

history = model.fit(train_generator,
                    steps_per_epoch=train_steps_per_epoch,
                    validation_data=val_generator,
                    validation_steps=val_steps_per_epoch,
                    epochs=epochs,
                    shuffle=True,
                    callbacks=[lr_reduce, early_stop, checkpoint],
                    verbose=1)

Epoch 1/100
  1/390 [..............................] - ETA: 3:14:36 - loss: 13.3235 - accuracy: 0.1250

KeyboardInterrupt: 